In [1]:
import pandas as pd

feature_matrix = pd.read_csv("/Users/Andria/Desktop/symsense-mit-challenge-2025-Jupyter-first/data/processed/flaredown/feature_matrix.csv")

foods_grouped = pd.read_csv("/Users/Andria/Desktop/symsense-mit-challenge-2025-Jupyter-first/data/processed/flaredown/foods_grouped.csv")
symptoms_grouped = pd.read_csv("/Users/Andria/Desktop/symsense-mit-challenge-2025-Jupyter-first/data/processed/flaredown/symptoms_grouped.csv")


In [ ]:
from sklearn.model_selection import train_test_split

X = feature_matrix.drop(
    columns=["user_id", "autoimmune_label",
             "ulcerative colitis", "rheumatoid arthritis", "lupus"]
)

y = feature_matrix["autoimmune_label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_cols = ["sex","country"]
ct = ColumnTransformer([
    ("onehot", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
], remainder="passthrough")

X_train_enc = ct.fit_transform(X_train)
X_test_enc = ct.transform(X_test)

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=2000)
clf.fit(X_train_enc, y_train)

y_pred = clf.predict(X_test_enc)
y_prob = clf.predict_proba(X_test_enc)[:,1]

from sklearn.metrics import roc_auc_score, f1_score
print("AUC-ROC:", roc_auc_score(y_test, y_prob))
print("F1 Score:", f1_score(y_test, y_pred))

KeyError: 'checkin_date'